# Compile the model

In [1]:
from keras import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers

Using TensorFlow backend.
/home/niraj/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
model = Sequential()
model.add(Flatten(input_shape=(7,7,512)))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [5]:
adam = optimizers.Adam(lr=1e-6)
# rmsprop = optimizers.RMSprop(lr=1e-6)

In [6]:
model.compile(optimizer= adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Import features from bcolz store

In [7]:
import bcolz

In [8]:
train_features = bcolz.open(rootdir='train_store', mode='r')
val_features = bcolz.open(rootdir='val_store', mode='r')

In [9]:
train_features.shape

(23000, 7, 7, 512)

In [10]:
val_features.shape

(2000, 7, 7, 512)

# Import labels from pickle file

In [11]:
import pickle

In [12]:
with open("train_labels.pkl", "rb") as f:
    train_labels = pickle.load(f)
f.close()

with open("val_labels.pkl", "rb") as f:
    val_labels = pickle.load(f)
f.close()

In [13]:
train_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [14]:
val_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

# Initialize keras callbacks

In [15]:
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger, EarlyStopping

In [16]:
tb = TensorBoard(log_dir='./logs', batch_size=64, write_images=True, write_graph=True)

In [17]:
mc = ModelCheckpoint('./models/weights.{epoch:02d}-{val_loss:.3f}.hdf5', save_best_only=True)

In [18]:
cl = CSVLogger('./CSV/baseline.csv', separator=',')

In [19]:
es = EarlyStopping(monitor='val_loss', patience=5)

# Fit the model to the features

In [ ]:
model.fit(train_features, 
          train_labels, 
          batch_size=64, 
          epochs=10, 
          callbacks = [tb, mc, cl, es],
          validation_data=(val_features, val_labels),
         )

Train on 23000 samples, validate on 2000 samples
Epoch 1/10
23000/23000 [==============================] - 29s 1ms/step - loss: 3.0423 - acc: 0.7345 - val_loss: 0.2612 - val_acc: 0.9610
Epoch 2/10
23000/23000 [==============================] - 27s 1ms/step - loss: 1.1483 - acc: 0.8916 - val_loss: 0.1956 - val_acc: 0.9725
Epoch 3/10
23000/23000 [==============================] - 27s 1ms/step - loss: 0.7919 - acc: 0.9243 - val_loss: 0.1892 - val_acc: 0.9780
Epoch 4/10
23000/23000 [==============================] - 27s 1ms/step - loss: 0.6591 - acc: 0.9376 - val_loss: 0.1817 - val_acc: 0.9780
Epoch 5/10
23000/23000 [==============================] - 26s 1ms/step - loss: 0.5152 - acc: 0.9509 - val_loss: 0.1809 - val_acc: 0.9790
Epoch 6/10
23000/23000 [==============================] - 26s 1ms/step - loss: 0.4622 - acc: 0.9551 - val_loss: 0.1793 - val_acc: 0.9800
Epoch 7/10
23000/23000 [==============================] - 25s 1ms/step - loss: 0.3967 - acc: 0.9632 - val_loss: 0.1832 - val_acc:

# TODO

* Try data augmentation
* Try adding dropout and batch normalization
* L2 Regualizer maybe?
* Submit